**Tabla de contenido**

- [The Dataset](#The-Dataset)
- [Multilingual Transformers](#Multilingual-Transformers)
- [Una mirada más cercana a la tokenización](#Una-mirada-mas-cercana-a-la-tokenizacion)
     - [The Tokenizer Pipeline](#The-Tokenizer-Pipeline)
     - [El tokenizador SentencePiece](#El-tokenizador-SentencePiece)

Hasta ahora en este libro hemos aplicado transformadores para resolver tareas de NLP en corpora en inglés, pero `¿qué haces cuando tus documentos están escritos en griego, swahili o klingon?` Un enfoque es buscar en el Hugging Face Hub un modelo de lenguaje preentrenado adecuado y ajustarlo para la tarea en cuestión. Sin embargo, estos modelos preentrenados tienden a existir solo para lenguajes de "alto recurso" como alemán, ruso o mandarín, donde hay mucho texto web disponible para el preentrenamiento. Otro desafío común surge cuando tu corpus es multilingüe: mantener múltiples modelos monolingües en producción no será divertido ni para ti ni para tu equipo de ingeniería.

Afortunadamente, existe una clase de transformadores multilingües que vienen al rescate. Al igual que BERT, estos modelos utilizan el modelado de lenguaje enmascarado como objetivo de preentrenamiento, pero se entrenan conjuntamente en textos en más de cien idiomas. Al preentrenarse en grandes corpus a través de muchos idiomas, estos transformadores multilingües permiten la transferencia cruzada de cero disparos. Esto significa que un modelo que se ajusta finamente a un idioma puede aplicarse a otros sin ningún entrenamiento adicional. ¡Esto también hace que estos modelos sean muy adecuados para el 'cambio de código', donde un hablante alterna entre dos o más idiomas o dialectos en el contexto de una sola conversación!

En este capítulo exploraremos cómo un único modelo de transformador llamado XLM-RoBERTa (introducido en el Capítulo 3) puede ser ajustado para realizar el reconocimiento de entidades nombradas (NER) en varios idiomas. Como vimos en el Capítulo 1, el NER es una tarea común de procesamiento del lenguaje natural que identifica entidades como personas, organizaciones o ubicaciones en el texto. Estas entidades pueden ser utilizadas para diversas aplicaciones, como obtener información de documentos de empresas, mejorar la calidad de los motores de búsqueda, o simplemente construir una base de datos estructurada a partir de un corpus.

Para este capítulo, supongamos que queremos realizar NER para un cliente con sede en Suiza, donde hay cuatro idiomas nacionales (siendo el inglés a menudo un puente entre ellos). Empecemos obteniendo un corpus multilingüe adecuado para este problema.

Nota: `la Zero-shot transfer`  o el `zero-shot learning` generalmente se refiere a la tarea de entrenar un modelo en un conjunto de etiquetas y luego evaluarlo en un conjunto diferente de etiquetas. En el contexto de los transformadores, el zero-shot learning también puede referirse a situaciones en las que un modelo de lenguaje como GPT-3 es evaluado en una tarea posterior en la que ni siquiera se ha ajustado.

# The Dataset

En este capítulo, utilizaremos un subconjunto del benchmark de Evaluación de Transferencia Multilingüe de Encoders (XTREME) llamado WikiANN o PAN-X. Este conjunto de datos consiste en artículos de Wikipedia en muchos idiomas, incluidos los cuatro idiomas más hablados en Suiza: alemán (62.9%), francés (22.9%), italiano (8.4%) e inglés (5.9%).

Cada artículo está anotado con etiquetas LOC (ubicación), PER (persona) y ORG (organización) en el formato "dentro-fuera-comienzo" (IOB2). En este formato, un prefijo B indica el comienzo de una entidad, y los tokens consecutivos que pertenecen a la misma entidad reciben un prefijo I-. Una etiqueta O indica que el token no pertenece a ninguna entidad. Por ejemplo, la siguiente oración:

 `Jeff Dean is a computer scientist at Google in California` 
 
 se etiquetaría en formato IOB2 como se muestra en la siguiente Tabla.

![Tabla](images/table4.png)

Para cargar uno de los subconjuntos de PAN-X en XTREME, necesitaremos saber qué configuración de conjunto de datos pasar a la función load_dataset(). Siempre que trabajes con un conjunto de datos que tiene múltiples dominios, puedes usar la función get_dataset_config_names() para averiguar qué subconjuntos están disponibles:

In [1]:
from datasets import get_dataset_config_names
xtreme_subsets = get_dataset_config_names("xtreme")
print(f"XTREME has {len(xtreme_subsets)} configurations")

README.md: 0.00B [00:00, ?B/s]

XTREME has 183 configurations


Vaya, ¡eso es un montón de configuraciones! Vamos a reducir la búsqueda buscando solo las configuraciones que comienzan con "PAN":

In [2]:
panx_subsets = [s for s in xtreme_subsets if s.startswith("PAN")]
panx_subsets[:3]

['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg']

De acuerdo, parece que hemos identificado la sintaxis de los subconjuntos de PAN-X: cada uno tiene un sufijo de dos letras que parece ser un código de idioma ISO 639-1. Esto significa que para cargar el corpus en alemán, pasamos el código al argumento name de load_dataset() de la siguiente manera:

In [ ]:
from datasets import load_dataset
load_dataset("xtreme", name="PAN-X.de") # Cargar un dataset PAN-X para un idioma

PAN-X.de/train-00000-of-00001.parquet:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

PAN-X.de/validation-00000-of-00001.parqu(…):   0%|          | 0.00/590k [00:00<?, ?B/s]

PAN-X.de/test-00000-of-00001.parquet:   0%|          | 0.00/588k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
})

Para crear un corpus suizo realista, muestreamos los corpora de alemán (de), francés (fr), italiano (it) y inglés (en) de PAN-X de acuerdo con sus proporciones habladas. Esto creará un desequilibrio lingüístico que es muy común en conjuntos de datos del mundo real, donde adquirir ejemplos etiquetados en una lengua minoritaria puede ser costoso debido a la falta de expertos en la materia que hablen con fluidez ese idioma. Este conjunto de datos desequilibrado simulará una situación común al trabajar en aplicaciones multilingües, y veremos cómo podemos construir un modelo que funcione en todos los idiomas.

Para llevar un registro de cada idioma, vamos a crear un defaultdict de Python que almacene el código del idioma como clave y un corpus PAN-X del tipo DatasetDict como valor:

In [ ]:
"""
Carga los datasets PAN-X para 4 idiomas (alemán, francés, italiano, inglés) y 
toma solo una fracción específica de los datos para balancearlos
"""


from collections import defaultdict
from datasets import DatasetDict

langs = ["de", "fr", "it", "en"]
fracs = [0.629, 0.229, 0.084, 0.059]
panx_ch = defaultdict(DatasetDict)

for lang, frac in zip(langs, fracs):
    # Load monolingual corpus
    ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
    for split in ds:
        panx_ch[lang][split] = (
        ds[split].shuffle(seed=0).select(range(int(frac * ds[split].num_rows)))
        )

PAN-X.fr/train-00000-of-00001.parquet:   0%|          | 0.00/837k [00:00<?, ?B/s]

PAN-X.fr/validation-00000-of-00001.parqu(…):   0%|          | 0.00/419k [00:00<?, ?B/s]

PAN-X.fr/test-00000-of-00001.parquet:   0%|          | 0.00/423k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

...

PAN-X.en/validation-00000-of-00001.parqu(…):   0%|          | 0.00/472k [00:00<?, ?B/s]

PAN-X.en/test-00000-of-00001.parquet:   0%|          | 0.00/472k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Aquí hemos utilizado el método shuffle() para asegurarnos de que no sesgamos accidentalmente nuestras divisiones de conjuntos de datos, mientras que select() nos permite reducir la muestra de cada corpus de acuerdo con los valores en fracs. Echemos un vistazo a cuántos ejemplos tenemos por idioma en los conjuntos de entrenamiento accediendo al atributo Dataset.num_rows:

In [5]:
import pandas as pd
pd.DataFrame({lang: [panx_ch[lang]["train"].num_rows] for lang in langs},index=["Number of training examples"])

,de,fr,it,en
Number of training examples,12580,4580,1680,1180


Por diseño, tenemos más ejemplos en alemán que en todos los demás idiomas combinados, por lo que lo usaremos como punto de partida para realizar una transferencia cruzada multilingüe de cero disparos al francés, italiano e inglés. Inspeccionemos uno de los ejemplos en el corpus en alemán:


In [6]:
element = panx_ch["de"]["train"][0]
for key, value in element.items():
    print(f"{key}: {value}")

tokens: ['2.000', 'Einwohnern', 'an', 'der', 'Danziger', 'Bucht', 'in', 'der', 'polnischen', 'Woiwodschaft', 'Pommern', '.']
ner_tags: [0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0]
langs: ['de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de']


Al igual que en nuestros encuentros previos con objetos Dataset, las claves de nuestro ejemplo corresponden a los nombres de las columnas de una tabla Arrow, mientras que los valores denotan las entradas en cada columna. En particular, vemos que la columna ner_tags corresponde al mapeo de cada entidad a un ID de clase. Esto es un poco críptico para el ojo humano, así que vamos a crear una nueva columna con las etiquetas familiares LOC, PER y ORG. Para hacer esto, lo primero a notar es que nuestro objeto Dataset tiene un atributo de características que especifica los tipos de datos subyacentes asociados con cada columna:

In [7]:
for key, value in panx_ch["de"]["train"].features.items():
    print(f"{key}: {value}")

tokens: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
ner_tags: Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)
langs: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)


La clase Sequence especifica que el campo contiene una lista de características, que en el caso de ner_tags corresponde a una lista de características ClassLabel. Seleccionemos esta característica del conjunto de entrenamiento de la siguiente manera:

In [8]:
tags = panx_ch["de"]["train"].features["ner_tags"].feature
print(tags)

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)


Podemos usar el método ClassLabel.int2str() que encontramos en el Capítulo 2 para crear una nueva columna en nuestro conjunto de entrenamiento con los nombres de clase para cada etiqueta. Usaremos el método map() para devolver un diccionario con la clave correspondiente al nuevo nombre de columna y el valor como una lista de nombres de clase:

In [9]:
def create_tag_names(batch):
    return {"ner_tags_str": [tags.int2str(idx) for idx in batch["ner_tags"]]}
panx_de = panx_ch["de"].map(create_tag_names)

Map:   0%|          | 0/12580 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

Ahora que tenemos nuestras etiquetas en un formato legible para humanos, veamos cómo se alinean los tokens y las etiquetas para el primer ejemplo en el conjunto de entrenamiento.

In [10]:
de_example = panx_de["train"][0]
pd.DataFrame([de_example["tokens"], de_example["ner_tags_str"]],
['Tokens', 'Tags'])

,0,1,2,3,4,5,6,7,8,9,10,11
Tokens,2.000,Einwohnern,an,der,Danziger,Bucht,in,der,polnischen,Woiwodschaft,Pommern,.
Tags,O,O,O,O,B-LOC,I-LOC,O,O,B-LOC,B-LOC,I-LOC,O


La presencia de las etiquetas LOC tiene sentido ya que la frase "2,000 Einwohnern an der Danziger Bucht in der polnischen Woiwodschaft Pommern" significa "2,000 habitantes en la bahía de Gdansk en la voivodía polaca de Pomerania" en inglés, y la bahía de Gdansk es una bahía en el mar Báltico, mientras que "voivodía" corresponde a un estado en Polonia.

Como una verificación rápida de que no tenemos ningún desequilibrio inusual en las etiquetas, calculemos las frecuencias de cada entidad en cada división:


In [ ]:
"""
Recorre cada partición del dataset (train, test, etc.) y cuenta cuántas veces 
aparece cada tipo de entidad (como LOC, ORG, PER) al detectar las etiquetas que empiezan por B- (inicio de entidad).
"""
from collections import Counter
split2freqs = defaultdict(Counter)
for split, dataset in panx_de.items():
    for row in dataset["ner_tags_str"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1
pd.DataFrame.from_dict(split2freqs, orient="index")

,LOC,ORG,PER
train,6186,5366,5810
validation,3172,2683,2893
test,3180,2573,3071


Esto se ve bien: las distribuciones de las frecuencias de PER, LOC y ORG son aproximadamente las mismas para cada división, por lo que los conjuntos de validación y prueba deberían proporcionar una buena medida de la capacidad de generalización de nuestro etiquetador NER. A continuación, veamos algunos transformadores multilingües populares y cómo se pueden adaptar para abordar nuestra tarea de NER.

# Multilingual Transformers

Los transformadores multilingües implican arquitecturas y procedimientos de entrenamiento similares a los de sus contrapartes monolingües, excepto que el corpus utilizado para el preentrenamiento consiste en documentos en muchos idiomas. Una característica notable de este enfoque es que, a pesar de no recibir información explícita para diferenciar entre los idiomas, las representaciones lingüísticas resultantes son capaces de generalizar bien entre idiomas para una variedad de tareas posteriores. En algunos casos, esta capacidad de realizar transferencia entre idiomas puede producir resultados que son competitivos con los de los modelos monolingües, lo que elude la necesidad de entrenar un modelo por cada idioma!

Para medir el progreso de la transferencia multilingüe para el reconocimiento de entidades nombradas (NER), los conjuntos de datos CoNLL-2002 y CoNLL-2003 se utilizan a menudo como referencia para inglés, neerlandés, español y alemán. Esta referencia consiste en artículos de noticias anotados con las mismas categorías LOC, PER y ORG que PAN-X, pero contiene una etiqueta adicional MISC para entidades diversas que no pertenecen a los tres grupos anteriores. Los modelos de transformadores multilingües suelen ser evaluados de tres maneras diferentes:

- `en`: Ajustar finamente los datos de entrenamiento en inglés y luego evaluar en el conjunto de pruebas de cada idioma.
- `each`: Ajustar y evaluar en datos de prueba monolingües para medir el rendimiento por idioma.
- `all`: Ajustar finamente en todos los datos de entrenamiento para evaluar en cada conjunto de pruebas de cada idioma.

Adoptaremos una estrategia de evaluación similar para nuestra tarea de NER, pero primero necesitamos seleccionar un modelo para evaluar. Uno de los primeros transformadores multilingües fue mBERT, que utiliza la misma arquitectura y objetivo de preentrenamiento que BERT, pero añade artículos de Wikipedia de muchos idiomas al corpus de preentrenamiento. Desde entonces, mBERT ha sido superado por XLM-RoBERTa (o XLM-R para abreviar), así que ese es el modelo que consideraremos en este capítulo.

Como vimos en el Capítulo 3, XLM-R utiliza solo MLM como un objetivo de preentrenamiento para 100 idiomas, pero se distingue por el enorme tamaño de su corpus de preentrenamiento en comparación con sus precursores: volcado de Wikipedia para cada idioma y 2.5 terabytes de datos de Common Crawl de la web. Este corpus es varios órdenes de magnitud más grande que los utilizados en modelos anteriores y proporciona un impulso significativo en la señal para idiomas de bajos recursos como el birmano y el swahili, donde solo existe un pequeño número de artículos de Wikipedia.

La parte del nombre del modelo denominada RoBERTa se refiere al hecho de que el enfoque de preentrenamiento es el mismo que el de los modelos monolingües de RoBERTa. Los desarrolladores de RoBERTa mejoraron varios aspectos de BERT, en particular eliminando por completo la tarea de predicción de la siguiente oración. XLM-R también elimina las incrustaciones de lenguaje usadas en XLM y utiliza SentencePiece para tokenizar los textos sin procesar directamente. Además de su naturaleza multilingüe, una diferencia notable entre XLM-R y RoBERTa es el tamaño de sus respectivos vocabularios: 250,000 tokens frente a 55,000!

XLM-R es una excelente opción para tareas de NLU multilingües. En la próxima sección, exploraremos cómo puede tokenizar de manera eficiente en varios idiomas.


# Una mirada mas cercana a la tokenizacion

En lugar de utilizar un tokenizador WordPiece, XLM-R utiliza un tokenizador llamado SentencePiece que se entrena en el texto en bruto de los cien idiomas. Para entender cómo se compara SentencePiece con WordPiece, carguemos los tokenizadores de BERT y XLM-R de la manera habitual con Transformers:

In [1]:
from transformers import AutoTokenizer

bert_model_name = "bert-base-cased"
xlmr_model_name = "xlm-roberta-base"

bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Al codificar una pequeña secuencia de texto, también podemos recuperar los tokens especiales que cada modelo utilizó durante el preentrenamiento:

In [2]:
text = "Jack Sparrow loves New York!"
bert_tokens = bert_tokenizer(text).tokens()
xlmr_tokens = xlmr_tokenizer(text).tokens()

## The Tokenizer Pipeline

Hasta ahora, hemos tratado la tokenización como una sola operación que transforma cadenas en enteros que podemos pasar a través del modelo. Esto no es del todo exacto, y si miramos más de cerca, podemos ver que en realidad es un pipeline de procesamiento completo que generalmente consiste en cuatro pasos:
1. `Normalization`: Este paso corresponde al conjunto de operaciones que aplicas a una cadena de texto sin procesar para hacerla "más limpia". Las operaciones comunes incluyen eliminar espacios en blanco y quitar caracteres acentuados. La normalización de Unicode es otra operación de normalización común aplicada por muchos tokenizadores para abordar el hecho de que a menudo existen varias maneras de escribir el mismo carácter. Esto puede hacer que dos versiones de la "misma" cadena (es decir, con la misma secuencia de caracteres abstractos) parezcan diferentes; los esquemas de normalización de Unicode como NFC, NFD, NFKC y NFKD reemplazan las diversas maneras de escribir el mismo carácter con formas estándar. Otro ejemplo de normalización es convertir a minúsculas. Si se espera que el modelo acepte y utilice solo caracteres en minúscula, esta técnica se puede usar para reducir el tamaño del vocabulario que requiere. Después de la normalización, nuestra cadena de ejemplo se vería como "¡jack sparrow ama nueva york!".
2. `Pretokenization`: Este paso divide un texto en objetos más pequeños que proporcionan un límite superior a lo que serán tus tokens al final del entrenamiento. Una buena manera de pensar en esto es que el pretokenizador dividirá tu texto en "palabras", y tus tokens finales serán partes de esas palabras. Para los idiomas que permiten esto (inglés, alemán y muchos idiomas indoeuropeos), las cadenas generalmente pueden dividirse en palabras mediante espacios en blanco y puntuación. Por ejemplo, este paso puede transformar nuestro ["jack", "sparrow", "loves", "new", "york", "!"];.Estas palabras son más simples de dividir en subpalabras con algoritmos de Byte-Pair Encoding (BPE) o Unigram en el siguiente paso de la tubería. Sin embargo, dividir en "palabras" no siempre es una operación trivial y determinista, o incluso una operación que tenga sentido. Por ejemplo, en idiomas como el chino, japonés o coreano, agrupar símbolos en unidades semánticas como las palabras indoeuropeas puede ser una operación no determinista con varios grupos igualmente válidos. En este caso, podría ser mejor no pretokenizar el texto y, en su lugar, utilizar una biblioteca específica para el idioma para la pretokenización.
3. `Tokenizer model`: Una vez que los textos de entrada están normalizados y pretokens, el tokenizador aplica un modelo de división de subpalabras sobre las palabras. Esta es la parte del proceso que necesita ser entrenada en su corpus (o que ha sido entrenada si está utilizando un tokenizador preentrenado). El papel del modelo es dividir las palabras en subpalabras para reducir el tamaño del vocabulario y tratar de disminuir el número de tokens fuera de vocabulario. Existen varios algoritmos de tokenización de subpalabras, incluyendo BPE, Unigram y WordPiece. Por ejemplo, nuestro ejemplo en funcionamiento podría verse como [jack, spa, rrow, loves, new, york, !] después de aplicar el modelo tokenizador. Tenga en cuenta que en este punto ya no tenemos una lista de cadenas sino una lista de enteros (ID de entrada); para mantener el ejemplo ilustrativo, hemos conservado las palabras pero hemos quitado las comillas para indicar la transformación.
4. `Postprocessing`:Este es el último paso del proceso de tokenización, en el cual se pueden aplicar algunas transformaciones adicionales a la lista de tokens; por ejemplo, agregar tokens especiales al principio o al final de la secuencia de entrada de índices de tokens. Por ejemplo, un tokenizador estilo BERT agregaría tokens de clasificación y separación: [CLS, jack, spa, rrow, ama, nueva, york, !, SEP]. Esta secuencia (recuerda que será una secuencia de números enteros, no los tokens que ves aquí) puede ser alimentada al modelo.

Volviendo a nuestra comparación de XLM-R y BERT, ahora entendemos que SentencePiece agrega "s y \s" en lugar de [CLS] y [SEP] en el paso de posprocesamiento (como convención, continuaremos usando [CLS] y [SEP] en las ilustraciones gráficas). Volvamos al tokenizador SentencePiece para ver qué lo hace especial.

## El tokenizador SentencePiece

El tokenizador SentencePiece se basa en un tipo de segmentación de subpalabras llamada Unigram y codifica cada texto de entrada como una secuencia de caracteres Unicode. Esta última característica es especialmente útil para corpora multilingües, ya que permite que SentencePiece sea agnóstico sobre acentos, puntuación y el hecho de que muchos idiomas, como el japonés, no tienen caracteres de espacio en blanco. Otra característica especial de SentencePiece es que el espacio en blanco se asigna al símbolo Unicode U+2581, o al carácter , también llamado el carácter de bloque inferior.

Esto permite que SentencePiece des-tokenice una secuencia sin ambigüedades y sin depender de pre-tokenizadores específicos del idioma. En nuestro ejemplo de la sección anterior, por ejemplo, podemos ver que WordPiece ha perdido la información de que no hay espacio en blanco entre "York" y "!". Por el contrario, SentencePiece preserva el espacio en blanco en el texto tokenizado, por lo que podemos convertir de nuevo al texto sin procesar sin ambigüedad:

In [3]:
"".join(xlmr_tokens).replace(u"\u2581", " ")

'<s> Jack Sparrow loves New York!</s>'